In [1]:
import sys
!{sys.executable} -m pip install --user twitter
from twitter import *
from IPython.display import HTML
from ipywidgets import interact

ACCESS_TOKEN = "988216913284255744-qBZJ3IAkrjBAPLaMRMjhkQrLtKIzn31"
ACCESS_TOKEN_SECRET = "WpcZY8dTZAQVWEJkZBlS8hMQaR9ckqOs8ErFML2x2bVAk"
CONSUMER_KEY = "sRKZDEri6WsOhDmJmxxbnLvyC"
CONSUMER_SECRET = "OJO5n71PvgNE6B6JLePAlRTyOlDoTFFpWziUEE4stu0pbNNLHo"

        
    
class twitUser:
    def __init__(self, name, screenName, followerCount, location, degrees, profileImageURL):
        self.name = name
        self.screenName = screenName
        self.followerCount = followerCount
        self.location = location
        self.degrees = degrees
        self.profileImageURL = profileImageURL

def calculateScore(twitUser, RTs, Hashtag55):
    score = twitUser.followerCount / 10 + 150 / twitUser.degrees + (100 * (getHashCount(twitUser, Hashtag55, RTs)))
    return score

class node:
    def __init__(self, twitUser, val):
        self.t = twitUser
        self.v = val

class maxHeap:
 # Constructor to initialize a heap
    def __init__(self):
        self.heap = []

    def size(self):
        return len(self.heap)

    #index of parent of node at index i
    def parent(self, i):
        return (i-1)//2

    #index of left child of node at index i
    def left(self, i):
        return 2*i + 1

    #index of right child of node at index i
    def right(self, i):
        return 2*i + 2

    def swap(self, parent, child):
        temp = self.heap[child]
        self.heap[child] = self.heap[parent]
        self.heap[parent] = temp

    # Pushes node to heap
    def push(self, node):
        i = self.size()
        self.heap.append(node)

        while (i > 0 and self.heap[self.parent(i)].v < self.heap[i].v ):
            self.swap(self.parent(i), i)
            i = self.parent(i)

    def pop(self):
        if (self.size() == 0):
            return None
        i = 0
        heapMax = self.heap[i]
        self.heap[i] = self.heap[self.size() - 1]
        del self.heap[self.size() - 1]
        while (self.left(i) < self.size()):
            if (self.right(i) < self.size()):
                if (self.heap[i].v < self.heap[self.left(i)].v or self.heap[i].v < self.heap[self.right(i)].v):
                    if (self.heap[self.left(i)].v > self.heap[self.right(i)].v):
                        self.swap(i, self.left(i))
                        i = self.left(i)
                    else:
                        self.swap(i, self.right(i))
                        i = self.right(i)
                else:
                    break
            else:
                if (self.heap[i].v < self.heap[self.left(i)].v):
                    self.swap(i, self.left(i))
                    i = self.left(i)
                else:
                    break

        return heapMax


    # Get the minimum element from the heap
    def getMax(self):
        return self.heap[0]

    def printHeap(self, count=10):
        i = 0
        sz = self.size()
        html = '''<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
.card {
    box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
    transition: 0.3s;
    width: 20%;
    float: left;
}

.card:hover {
    box-shadow: 0 8px 16px 0 rgba(0,0,0,0.2);
}

.container {
    padding: 2px 16px;
}
</style>
</head><body>'''
        while(i < count and i < sz):
            node = self.pop()
            html += '''
            <div class="card">
              <img src="{}" alt="Avatar" style="width:100px; length:100px; padding-left:60px;">
              <div class="container">
                <h4><b>{}</b></h4> 
                <p>@{}</p> 
                <p>Followers: {}</p>
                <p>Score: {}</p>
              </div>
            </div>
            '''.format(node.t.profileImageURL, node.t.name, node.t.screenName, node.t.followerCount, node.v)
            i = i + 1
        html += '</body>'
        display(HTML(html))


    def delHeap(self):
        del self.heap[:]
        return None

def getHashCount(user, hashtag1, retweets): #get number of tweets with a given hashtag
    HashCount = 0
    tester1 = t.statuses.user_timeline(screen_name=user, count = 3)
    for TweetList in tester1:# I believe count is number of tweets
        #These are just checks right now above
        if TweetList["is_quote_status"] and not retweets: #should we look for retweets too?
            continue
        for HashList in TweetList["entities"]["hashtags"]:
            if HashList["text"].lower() == hashtag1.lower(): #match
                HashCount = HashCount + 1
    return HashCount

def getFollowers(MaxHeap, user, degree, maxFollowers, RT=False, Hashtag=""): #recursive function that access followers
    if degree == 3:
        return 0
    x = t.followers.list(screen_name=user, count = 200) #dictionary containing list of followers
    while True:
        for i in range(0,len(x['users'])): #iterate through list of followers
            follower = x['users'][i] #access follower object
            if follower['followers_count'] > maxFollowers: # don't want users with too many followers b/c unrealistic
                continue
            twitFollower = twitUser(follower['name'], follower['screen_name'], follower['followers_count'], follower['location'], degree, follower['profile_image_url'])
            twitNode = node(twitFollower, calculateScore(twitFollower, RT, Hashtag))
            heap.push(twitNode)

            if not follower['protected'] and not follower['verified'] and follower['followers_count'] < 2000: #recursively get follower's followers if account is not private
                getFollowers(MaxHeap, follower['screen_name'], degree + 1, maxFollowers, RT, Hashtag)
        cursor = x['next_cursor'] #get next page
        if cursor == 0: #no next page
            break
        x = t.followers.list(screen_name=user, cursor = cursor, count = 200) #turn to next page



# Put in token, token_key, con_secret, con_secret_key
t = Twitter(auth=OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET))

heap = maxHeap()
getFollowers(heap, "dijkstra_edsger", 1, 2000, True, Hashtag="gbr")
heap.printHeap(10)


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
